互评作业二: 频繁模式与关联规则挖掘

作业说明：

使用的数据集：Microsoft 资讯推荐:  https://learn.microsoft.com/zh-cn/azure/open-datasets/dataset-microsoft-news?tabs=azureml-opendatasets

代码地址：https://github.com/truth-z/DM2

In [1]:
import csv
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf

一.数据集处理

读取tsv数据集文件，将两个表behaviors.tsv、news.tsv关于新闻id做拼接，选取其中的历史记录、曝光数、新闻类别来进行数据挖掘。

In [2]:
# 读取tsv文件
# path 为数据集路径
path1 = 'MINDsmall_train\\behaviors.tsv'
path2 = 'MINDsmall_train\\news.tsv'

new_to_cate = {}    # 新闻序列 -> 新闻子类别
with open(path2, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    # 按行处理数据
    for row in reader:
        if row[0] not in new_to_cate:
            new_to_cate[row[0]] = row[2]
print(len(new_to_cate))

data = []
with open(path1, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        # 按行处理数据
        lis = []
        # print(row)

        # 处理时间
        hour = int(row[2].split(' ')[1].split(':')[0])
        t = row[2].split(' ')[-1]
        if t == 'PM':
            hour += 12
        if hour < 9:
            lis.append('hour_0_9')
        elif hour < 17:
            lis.append('hour_9_17')
        else:
            lis.append('hour_17_24')

        new_list = row[3].split(' ')
        for i in new_list:
            if i == '':
                break
            if new_to_cate[i] not in lis:
                lis.append(new_to_cate[i])
        # print(new_list, list)
        # print(list)

        # 处理关联新闻类别
        for i in row[4].split(' '):
            new_id = i.split('-')[0]
            if i.split('-')[-1] == '1':
                if new_to_cate[new_id] not in lis:
                    lis.append(new_to_cate[new_id])
        # print(list)

        data.append(list(lis))
        # print(data)

N = len(data)
print(len(data))
for i in range(5):
    print(data[i])

51282
156965
['hour_9_17', 'tvnews', 'baseball_mlb', 'newscrime', 'football_ncaa', 'lifestylebuzz', 'movienews', 'newspolitics', 'football_nfl']
['hour_17_24', 'newscrime', 'newsus', 'markets', 'travelnews', 'newsworld', 'finance-companies', 'boxing', 'basketball_nba', 'lifestyle-news-feature', 'science', 'weathertopstories', 'causes-environment', 'restaurantsandnews', 'musicnews', 'movies-celebrity', 'lifestylebuzz', 'health-news', 'traveltips', 'newsopinion', 'foodnews', 'football_nfl']
['hour_0_9', 'lifestylebuzz', 'lifestylehomeandgarden', 'newsus', 'football_nfl', 'tvvideos', 'weathertopstories', 'baseball_mlb', 'newstrends', 'shop-holidays', 'finance-homesandpropertysection', 'newsphotos', 'autosnews', 'voices']
['hour_0_9', 'tv-celebrity', 'baseball_mlb', 'markets', 'basketball_nba', 'lifestylebuzz', 'viral', 'football_ncaa', 'music-celebrity']
['hour_9_17', 'autosownership', 'travelnews', 'weathertopstories', 'weightloss', 'autosresearchguides']


二.关联关系挖掘

In [3]:
# 输入格式转换
int_to_str = {}     # 整数编码 —> 字符串
str_to_int= {}      # 字符串 -> 整数编码
idx = 0
data_int = []
for i in range(len(data)):
    tmp = []
    for col in range(len(data[i])):
        if data[i][col] in str_to_int:
            tmp.append(str_to_int[data[i][col]])
        else:
            int_to_str[idx] = data[i][col]
            str_to_int[data[i][col]] = idx
            tmp.append(idx)
            idx += 1
    # print(tmp)
    # print()
    data_int.append(tmp)
for i in range(5):
    print(data[i])
    print(data_int[i])
    print()

['hour_9_17', 'tvnews', 'baseball_mlb', 'newscrime', 'football_ncaa', 'lifestylebuzz', 'movienews', 'newspolitics', 'football_nfl']
[0, 1, 2, 3, 4, 5, 6, 7, 8]

['hour_17_24', 'newscrime', 'newsus', 'markets', 'travelnews', 'newsworld', 'finance-companies', 'boxing', 'basketball_nba', 'lifestyle-news-feature', 'science', 'weathertopstories', 'causes-environment', 'restaurantsandnews', 'musicnews', 'movies-celebrity', 'lifestylebuzz', 'health-news', 'traveltips', 'newsopinion', 'foodnews', 'football_nfl']
[9, 3, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 5, 24, 25, 26, 27, 8]

['hour_0_9', 'lifestylebuzz', 'lifestylehomeandgarden', 'newsus', 'football_nfl', 'tvvideos', 'weathertopstories', 'baseball_mlb', 'newstrends', 'shop-holidays', 'finance-homesandpropertysection', 'newsphotos', 'autosnews', 'voices']
[28, 5, 29, 10, 8, 30, 19, 2, 31, 32, 33, 34, 35, 36]

['hour_0_9', 'tv-celebrity', 'baseball_mlb', 'markets', 'basketball_nba', 'lifestylebuzz', 'viral', 'football_ncaa'

频繁项集挖掘

In [4]:
result_supp = dict(oaf.frequent_itemsets(data_int, .3)) #支持度
print(result_supp)
print(len(result_supp))


{frozenset({0}): 68647, frozenset({1}): 48725, frozenset({3}): 84422, frozenset({5}): 80711, frozenset({3, 5}): 52786, frozenset({7}): 82116, frozenset({3, 7}): 51456, frozenset({5, 7}): 48612, frozenset({8}): 75954, frozenset({10}): 118108, frozenset({0, 10}): 50902, frozenset({10, 3}): 73634, frozenset({10, 5}): 68581, frozenset({10, 3, 5}): 48687, frozenset({10, 7}): 70419, frozenset({10, 3, 7}): 47899, frozenset({8, 10}): 62630, frozenset({11}): 53088, frozenset({10, 11}): 47619, frozenset({12}): 51314, frozenset({13}): 78747, frozenset({3, 13}): 54120, frozenset({13, 5}): 50026, frozenset({13, 7}): 52047, frozenset({10, 13}): 69621, frozenset({10, 3, 13}): 50702, frozenset({10, 13, 7}): 48324, frozenset({14}): 61472, frozenset({10, 14}): 54086, frozenset({23}): 63088, frozenset({10, 23}): 54509, frozenset({27}): 51886, frozenset({28}): 54739, frozenset({37}): 85686, frozenset({3, 37}): 56368, frozenset({37, 5}): 55516, frozenset({37, 7}): 50822, frozenset({8, 37}): 47716, frozense

In [5]:
# [频繁项，支持度]
items = []
for i in result_supp:
    # print(i)
    tmp = []
    for j in i:
        tmp.append(int_to_str[j])
    items.append([tmp, round(result_supp[i]/N, 4)])
    # print(items[-1])

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(items, columns=['频繁项集', '支持度']) 
df = df.sort_values('支持度',ascending=False)
df.index = range(len(df))
print(df)

                                     频繁项集     支持度
0                                [newsus]  0.7524
1                          [tv-celebrity]  0.5459
2                             [newscrime]  0.5378
3                          [newspolitics]  0.5231
4                         [lifestylebuzz]  0.5142
5                             [newsworld]  0.5017
6                          [football_nfl]  0.4839
7                     [newsus, newscrime]  0.4691
8                  [newsus, tv-celebrity]  0.4604
9                  [newsus, newspolitics]  0.4486
10                    [newsus, newsworld]  0.4435
11                            [hour_9_17]  0.4373
12                [newsus, lifestylebuzz]  0.4369
13                     [movies-celebrity]  0.4019
14                 [football_nfl, newsus]  0.3990
15                    [finance-companies]  0.3916
16              [newscrime, tv-celebrity]  0.3591
17          [tv-celebrity, lifestylebuzz]  0.3537
18                             [hour_0_9]  0.3487


关联规则挖掘

In [6]:
result_rules = oaf.association_rules(result_supp, .5)   #置信度
result_rules = list(result_rules)
print(len(result_rules))
result_rules

75


[(frozenset({3, 5}), frozenset({10}), 48687, 0.9223468343879059),
 (frozenset({5}), frozenset({3, 10}), 48687, 0.6032263260274312),
 (frozenset({3}), frozenset({5, 10}), 48687, 0.5767098623581531),
 (frozenset({5, 10}), frozenset({3}), 48687, 0.7099196570478704),
 (frozenset({3, 10}), frozenset({5}), 48687, 0.6612027052720211),
 (frozenset({3, 7}), frozenset({10}), 47899, 0.9308729788557214),
 (frozenset({7}), frozenset({3, 10}), 47899, 0.5833089775439622),
 (frozenset({3}), frozenset({7, 10}), 47899, 0.5673758025159319),
 (frozenset({7, 10}), frozenset({3}), 47899, 0.6801999460372911),
 (frozenset({3, 10}), frozenset({7}), 47899, 0.6505011271966754),
 (frozenset({3, 13}), frozenset({10}), 50702, 0.9368440502586844),
 (frozenset({13}), frozenset({3, 10}), 50702, 0.6438594486139154),
 (frozenset({10, 13}), frozenset({3}), 50702, 0.7282572786946467),
 (frozenset({3, 10}), frozenset({13}), 50702, 0.6885677811880382),
 (frozenset({3}), frozenset({10, 13}), 50702, 0.6005780483760157),
 (fro

In [7]:
# [规则前项，规则后项，支持度，置信度]
Rules = []
for i in result_rules:
    tmp1 = []
    tmp2 = []
    for j in i[0]:   
        tmp1.append(int_to_str[j])
    for j in i[1]:
        tmp2.append(int_to_str[j])
    Rules.append([tmp1, tmp2, round(i[2]/N, 4), round(i[3], 4)])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(Rules, columns=['关联规则前项', '关联规则后项', '支持度', '置信度']) 
df = df.sort_values('置信度',ascending=False)
df.index = range(len(df))
print(df)

                           关联规则前项                   关联规则后项     支持度     置信度
0          [newscrime, newsworld]                 [newsus]  0.3230  0.9368
1       [newscrime, newspolitics]                 [newsus]  0.3052  0.9309
2       [newsworld, tv-celebrity]                 [newsus]  0.3001  0.9291
3       [newsworld, newspolitics]                 [newsus]  0.3079  0.9285
4      [newscrime, lifestylebuzz]                 [newsus]  0.3102  0.9223
5       [newscrime, tv-celebrity]                 [newsus]  0.3296  0.9177
6   [lifestylebuzz, tv-celebrity]                 [newsus]  0.3182  0.8998
7                       [markets]                 [newsus]  0.3034  0.8970
8                     [newsworld]                 [newsus]  0.4435  0.8841
9             [finance-companies]                 [newsus]  0.3446  0.8798
10                    [newscrime]                 [newsus]  0.4691  0.8722
11             [movies-celebrity]                 [newsus]  0.3473  0.8640
12                 [newsp

In [8]:
results = list(oaf.rules_stats(result_rules, result_supp, len(data_int)))

resultsRules = []
for i in results:
    tmp1 = []
    tmp2 = []
    for j in i[0]:  
        tmp1.append(int_to_str[j])
    for j in i[1]:
        tmp2.append(int_to_str[j])
    resultsRules.append([tmp1, tmp2, round(i[2]/N, 4), round(i[3], 4), round(i[6], 4), round(i[7], 4)])
df = pd.DataFrame(resultsRules, columns=['关联规则前项', '关联规则后项', '支持度', '置信度', 'lift', 'leverage']) 
df = df.sort_values('lift',ascending=False)
df.index = range(len(df))
df

,关联规则前项,关联规则后项,支持度,置信度,lift,leverage
0,[tv-celebrity],[movies-celebrity],0.3063,0.5610,1.3959,0.0869
1,[movies-celebrity],[tv-celebrity],0.3063,0.7620,1.3959,0.0869
2,[newsworld],"[newsus, newscrime]",0.3230,0.6439,1.3725,0.0877
3,"[newsus, newscrime]",[newsworld],0.3230,0.6886,1.3725,0.0877
4,"[newsus, newspolitics]",[newsworld],0.3079,0.6862,1.3679,0.0828
5,[newsworld],"[newsus, newspolitics]",0.3079,0.6137,1.3679,0.0828
6,"[newsus, newsworld]",[newscrime],0.3230,0.7283,1.3540,0.0845
7,[newscrime],"[newsus, newsworld]",0.3230,0.6006,1.3540,0.0845
8,"[newsus, tv-celebrity]",[lifestylebuzz],0.3182,0.6913,1.3444,0.0815
9,[lifestylebuzz],"[newsus, tv-celebrity]",0.3182,0.6189,1.3444,0.0815


三.模式命名

新闻类别之间的模式：根据历史记录和曝光数的新闻类别来关联当前的新闻，挖掘在点击一篇新闻后，用户大概率会点击那一部分类别的新闻。



四.可视化

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(items, columns=['频繁项集', '支持度']) 
df = df.sort_values('支持度',ascending=False)
df.index = range(len(df))
print('频繁项集：')
print(df)

频繁项集：
                                     频繁项集     支持度
0                                [newsus]  0.7524
1                          [tv-celebrity]  0.5459
2                             [newscrime]  0.5378
3                          [newspolitics]  0.5231
4                         [lifestylebuzz]  0.5142
5                             [newsworld]  0.5017
6                          [football_nfl]  0.4839
7                     [newsus, newscrime]  0.4691
8                  [newsus, tv-celebrity]  0.4604
9                  [newsus, newspolitics]  0.4486
10                    [newsus, newsworld]  0.4435
11                            [hour_9_17]  0.4373
12                [newsus, lifestylebuzz]  0.4369
13                     [movies-celebrity]  0.4019
14                 [football_nfl, newsus]  0.3990
15                    [finance-companies]  0.3916
16              [newscrime, tv-celebrity]  0.3591
17          [tv-celebrity, lifestylebuzz]  0.3537
18                             [hour_0_9]  0

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(Rules, columns=['关联规则前项', '关联规则后项', '支持度', '置信度']) 
df = df.sort_values('置信度',ascending=False)
df.index = range(len(df))
print("关联规则")
print(df)

关联规则
                           关联规则前项                   关联规则后项     支持度     置信度
0          [newscrime, newsworld]                 [newsus]  0.3230  0.9368
1       [newscrime, newspolitics]                 [newsus]  0.3052  0.9309
2       [newsworld, tv-celebrity]                 [newsus]  0.3001  0.9291
3       [newsworld, newspolitics]                 [newsus]  0.3079  0.9285
4      [newscrime, lifestylebuzz]                 [newsus]  0.3102  0.9223
5       [newscrime, tv-celebrity]                 [newsus]  0.3296  0.9177
6   [lifestylebuzz, tv-celebrity]                 [newsus]  0.3182  0.8998
7                       [markets]                 [newsus]  0.3034  0.8970
8                     [newsworld]                 [newsus]  0.4435  0.8841
9             [finance-companies]                 [newsus]  0.3446  0.8798
10                    [newscrime]                 [newsus]  0.4691  0.8722
11             [movies-celebrity]                 [newsus]  0.3473  0.8640
12                 [


五.挖掘结果分析


根据频繁项集用户点击的新闻类别排前三的分别是newsus、tv-celebrity、newsscrime，分别占比0.7524、0.5459、05378。同时，从频繁项集关联表可以看出用户点击的新闻类别组合排前三的分别是[newsus, newscrime]、[newsus, tv-celebrity]、[newsus, newspolitics]，分别占比0.4691、0.4604、0.4486。

根据挖掘的关联规则可以看出，在点击[newscrime, newsworld]新闻类别的用户里，有0.9368的用户继续点击了[newsus]新闻类别。在点击[newscrime, newspolitics]新闻类别的用户里，有0.9309的用户继续点击了[newsus]新闻类别。在点击[newsworld, tv-celebrity]新闻类别的用户里，有0.9291的用户继续点击了[newsus]新闻类别。